# Fortinet VPN Logs Analysis

This notebook helps to deal with VPN logs from Fortinet firewalls.
Although data conversion can be used for all types of "key-value" logs, data processing will not work with another type of log due to the column names.

## Logs conversion to csv

Use of multiprocessing to convert Fortinet firewall log files with non csv format to csv.

In [2]:
def read_files_folder(path):
    from os import walk
    files = []
    for (dirpath, dirnames, filenames) in walk(path):
        for filename in filenames:
            if filename.endswith("log"):
                file_path = dirpath + "\\" + filename
                files.append(file_path)
    return files

In [4]:
%%writefile process_file.py

def load_log(log_string_list):
    import re
    import pandas as pd
    pattern = re.compile('(\w+)=(?:"([^"]*)"|(\S*))')
    
    lines_s = pd.Series(log_string_list)
    data = lines_s.str.extractall(pattern)
    data.index = data.index.get_level_values(0)
    data[1] = data[1].fillna(data[2])
    return data.pivot(index=None, columns=0, values=1)

def process_file(file):
    import pandas as pd
    from os import path
    
    output_folder = r"C:\logs_vpn_out"
    first = True
    with open(file, "r") as f:
        if first:
            log_df = load_log(f.readlines())
            first = False
        else:
            t_df = load_log(f.readlines())
            log_df = log_df.append(t_df)
            del t_df
    file_name = path.basename(file)
    file_path = output_folder + "\\" + file_name + ".csv"
    log_df.to_csv(file_path)
    
    del log_df

Overwriting process_file.py


In [5]:
%%time
from multiprocessing import Pool
import process_file

if __name__ ==  '__main__': 
    num_processors = 12
    p=Pool(processes = num_processors)
    logs_folder = r"C:\logs_vpn_forti"

    files = read_files_folder(logs_folder)

    p.map(process_file.process_file, files)
    
    p.close()
    p.join()

Wall time: 5min 33s


## Dealing with different columns in files

In [32]:
import glob
import d6tstack.combine_csv

cfg_fnames = list(glob.glob(r'C:\logs_vpn_out\*.csv'))
c = d6tstack.combine_csv.CombinerCSV(cfg_fnames)

# check columns
print('all equal',c.is_all_equal())
print('')
d = c.is_column_present()

sniffing columns ok
all equal False



In [ ]:
fnames = d6tstack.combine_csv.CombinerCSV(cfg_fnames).to_csv_align(output_dir=r'C:\logs_vpn_out_fix')

## Reading log files with Dask from disk 

In [59]:
import dask.dataframe as dd
import pandas as pd

In [61]:
%%time
df = dd.read_csv(r'C:\logs_vpn_out_fix\*.csv',dtype={'error_num': 'object',
        'group': 'object','locport': 'object','peer_notif': 'object','reason': 'object','remport': 'object',
        'seq': 'object','spi': 'object','tunnelip': 'object','tunneltype': 'object','user': 'object',
        'desc': 'object','espauth': 'object','esptransform': 'object','in_spi': 'object','out_spi': 'object',
        'rcvdbyte': 'float','stage': 'object','tunnelid': 'object','Unnamed: 0': 'object','assignip': 'object',
        'dst_host': 'object','duration': 'float','eventtime': 'object','logid': 'object','nextstat': 'object',
        'phase2_name': 'object','sentbyte': 'float','exch': 'object','version': 'object'})

Wall time: 53 ms


## Researching

# Useful functions

In [63]:
# MaxMind ASN
import geoip2.database
def get_asn(ip):
    try:
        x = reader.asn(ip)
    except:
        return pd.np.nan
    try:
        return x.autonomous_system_organization if x else pd.np.nan
    except KeyError:
        return pd.np.nan

# MaxMind Country
def get_country(ip):
    try:
        x = reader.country(ip)
    except:
        return pd.np.nan
    try:
        return x.country.name if x else pd.np.nan
    except KeyError:
        return pd.np.nan

In [64]:
# Tor Nodes List
import requests
nodes = requests.request(method='GET', url="https://www.dan.me.uk/torlist/") #30 minutes limit
nodes_list = nodes.text.split("\n")

def is_tor_node(ip):
    if ip in nodes_list:
        return True
    else:
        return False

In [65]:
def abuseipdb_req(ip):
    import json
    try:
        key = <api_key>
        url = 'https://api.abuseipdb.com/api/v2/check'
        querystring = {
            'ipAddress': ip,
            'maxAgeInDays': '90'
        }
        headers = {
            'Accept': 'application/json',
            'Key': key
        }
        response = requests.request(method='GET', url=url, headers=headers, params=querystring)
        decodedResponse = json.loads(response.text)
    except e:
        print(e)
        return pd.np.nan
    try:
        return decodedResponse["data"]["abuseConfidenceScore"] if decodedResponse else pd.np.nan
    except KeyError:
        return pd.np.nan

In [66]:
# Convert bytes to human readable
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

## Searching for information exfiltration

In [68]:
%%time
df_exf = df[["remip", "tunnelid", "sentbyte"]].groupby(["remip","tunnelid"]).max().compute()

Wall time: 1min 29s


In [114]:
df_exf.reset_index(inplace=True)

In [115]:
df_exf["h_sentbyte"] = df_exf.sentbyte.apply(sizeof_fmt)

In [122]:
df_ext_10 = df_exf.nlargest(10, "sentbyte").sort_values(by="sentbyte", ascending=False).reset_index()

In [184]:
df_ext_10

,level_0,index,remip,tunnelid,sentbyte,h_sentbyte
0,495,495,103.6.139.246,2973221928.0,3.470560e+12,3.2TiB
1,1,1,103.6.139.246,3054928288.0,1.646262e+12,1.5TiB
2,164,164,191.101.209.126,1826793719.0,3.506824e+10,32.7GiB
3,214,214,191.101.209.72,1826793700.0,2.084387e+10,19.4GiB
4,291,291,191.101.209.72,1826793661.0,1.833587e+10,17.1GiB
5,361,361,115.134.131.154,3068782335.0,1.560168e+10,14.5GiB
6,293,293,171.99.130.158,3069050613.0,8.586856e+09,8.0GiB
7,37,37,210.61.150.144,3067193495.0,7.414605e+09,6.9GiB
8,32,32,202.188.229.70,3027033592.0,5.857543e+09,5.5GiB
9,13,13,122.103.241.185,3069552181.0,4.883923e+09,4.5GiB


In [124]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe'

In [127]:
fig = px.bar(df_ext_10.iloc[2:8], y="remip", x="sentbyte", title='Destiantion IPs by data sent', orientation='h')
fig.show()

### Data mining

In [129]:
%%time
## Add ASN
with geoip2.database.Reader(r'C:\maxmind\GeoLite2-ASN.mmdb') as reader:
    unique_ips = df_exf['remip'].unique()
    unique_ips = pd.Series(unique_ips, index = unique_ips)
    df_exf['ASN'] = df_exf['remip'].map(unique_ips.apply(get_asn))

Wall time: 34.1 ms


In [132]:
df_exf.nlargest(10, "sentbyte")

,index,remip,tunnelid,sentbyte,h_sentbyte,ASN
495,495,103.6.139.246,2973221928.0,3.470560e+12,3.2TiB,HKBN Enterprise Solutions HK Limited
1,1,103.6.139.246,3054928288.0,1.646262e+12,1.5TiB,HKBN Enterprise Solutions HK Limited
164,164,191.101.209.126,1826793719.0,3.506824e+10,32.7GiB,GTT Communications Inc.
214,214,191.101.209.72,1826793700.0,2.084387e+10,19.4GiB,GTT Communications Inc.
291,291,191.101.209.72,1826793661.0,1.833587e+10,17.1GiB,GTT Communications Inc.
361,361,115.134.131.154,3068782335.0,1.560168e+10,14.5GiB,"TM Net, Internet Service Provider"
293,293,171.99.130.158,3069050613.0,8.586856e+09,8.0GiB,"TRUE INTERNET Co.,Ltd."
37,37,210.61.150.144,3067193495.0,7.414605e+09,6.9GiB,Data Communication Business Group
32,32,202.188.229.70,3027033592.0,5.857543e+09,5.5GiB,"TM Net, Internet Service Provider"
13,13,122.103.241.185,3069552181.0,4.883923e+09,4.5GiB,ARTERIA Networks Corporation


In [133]:
%%time
## Add country
with geoip2.database.Reader(r'C:\maxmind\GeoLite2-Country.mmdb') as reader:
    unique_ips = df_exf['remip'].unique()
    unique_ips = pd.Series(unique_ips, index = unique_ips)
    df_exf['remip_country'] = df_exf['remip'].map(unique_ips.apply(get_country))

Wall time: 44.9 ms


In [134]:
df_exf.nlargest(10, "sentbyte")

,index,remip,tunnelid,sentbyte,h_sentbyte,ASN,remip_country
495,495,103.6.139.246,2973221928.0,3.470560e+12,3.2TiB,HKBN Enterprise Solutions HK Limited,Hong Kong
1,1,103.6.139.246,3054928288.0,1.646262e+12,1.5TiB,HKBN Enterprise Solutions HK Limited,Hong Kong
164,164,191.101.209.126,1826793719.0,3.506824e+10,32.7GiB,GTT Communications Inc.,United Kingdom
214,214,191.101.209.72,1826793700.0,2.084387e+10,19.4GiB,GTT Communications Inc.,United Kingdom
291,291,191.101.209.72,1826793661.0,1.833587e+10,17.1GiB,GTT Communications Inc.,United Kingdom
361,361,115.134.131.154,3068782335.0,1.560168e+10,14.5GiB,"TM Net, Internet Service Provider",Malaysia
293,293,171.99.130.158,3069050613.0,8.586856e+09,8.0GiB,"TRUE INTERNET Co.,Ltd.",Thailand
37,37,210.61.150.144,3067193495.0,7.414605e+09,6.9GiB,Data Communication Business Group,Taiwan
32,32,202.188.229.70,3027033592.0,5.857543e+09,5.5GiB,"TM Net, Internet Service Provider",Malaysia
13,13,122.103.241.185,3069552181.0,4.883923e+09,4.5GiB,ARTERIA Networks Corporation,Japan


In [157]:
df_ext_map = df_exf.sort_values(by="sentbyte", ascending=False).iloc[2:]

In [158]:
df_ext_map = df_ext_map[["sentbyte", "remip_country"]].groupby("remip_country").sum().reset_index()

In [171]:
import plotly as py
import plotly.graph_objs as go
import pandas as pd

data = dict (
    type = 'choropleth',
    locations = df_ext_map['remip_country'],
    locationmode='country names',
    colorscale = 'blugrn',
    z=df_ext_map['sentbyte'])

map = go.Figure(data=[data])
py.offline.plot(map)

'temp-plot.html'

### Search for Tor nodes communications

In [174]:
%%time
unique_remip = df_exf["remip"].unique()
unique_remip = pd.Series(unique_remip, index = unique_remip)
df_exf['is_tor_node'] = df_exf['remip'].map(unique_remip.apply(is_tor_node))

Wall time: 18 ms


In [176]:
df_exf[df_exf["is_tor_node"]]

,index,remip,tunnelid,sentbyte,h_sentbyte,ASN,remip_country,is_tor_node
189,189,185.220.100.247,0.0,NaN,nanYiB,F3 Netze e.V.,Germany,True
196,196,45.129.56.200,0.0,NaN,nanYiB,31173 Services AB,Denmark,True
210,210,185.220.100.244,0.0,NaN,nanYiB,F3 Netze e.V.,Germany,True
211,211,185.220.101.129,0.0,NaN,nanYiB,Cia Triad Security LLC,Germany,True


In [173]:
df_exf.nlargest(10, "sentbyte")

,index,remip,tunnelid,sentbyte,h_sentbyte,ASN,remip_country,is_tor_node
495,495,103.6.139.246,2973221928.0,3.470560e+12,3.2TiB,HKBN Enterprise Solutions HK Limited,Hong Kong,False
1,1,103.6.139.246,3054928288.0,1.646262e+12,1.5TiB,HKBN Enterprise Solutions HK Limited,Hong Kong,False
164,164,191.101.209.126,1826793719.0,3.506824e+10,32.7GiB,GTT Communications Inc.,United Kingdom,False
214,214,191.101.209.72,1826793700.0,2.084387e+10,19.4GiB,GTT Communications Inc.,United Kingdom,False
291,291,191.101.209.72,1826793661.0,1.833587e+10,17.1GiB,GTT Communications Inc.,United Kingdom,False
361,361,115.134.131.154,3068782335.0,1.560168e+10,14.5GiB,"TM Net, Internet Service Provider",Malaysia,False
293,293,171.99.130.158,3069050613.0,8.586856e+09,8.0GiB,"TRUE INTERNET Co.,Ltd.",Thailand,False
37,37,210.61.150.144,3067193495.0,7.414605e+09,6.9GiB,Data Communication Business Group,Taiwan,False
32,32,202.188.229.70,3027033592.0,5.857543e+09,5.5GiB,"TM Net, Internet Service Provider",Malaysia,False
13,13,122.103.241.185,3069552181.0,4.883923e+09,4.5GiB,ARTERIA Networks Corporation,Japan,False


### AbuseIPDB score

In [181]:
%%time
unique_ips = df_exf["remip"].unique()
unique_ips = pd.Series(unique_ips, index = top10)
df_exf['remip_abuse'] = df_exf['remip'].map(unique_ips.apply(abuseipdb_req))

Wall time: 2min 22s


In [182]:
df_exf.nlargest(10, "remip_abuse")

,index,remip,tunnelid,sentbyte,h_sentbyte,ASN,remip_country,is_tor_node,remip_abuse
16,16,162.142.125.39,0.0,NaN,nanYiB,CENSYS-ARIN-01,United States,False,100
31,31,192.241.210.30,0.0,NaN,nanYiB,DIGITALOCEAN-ASN,United States,False,100
49,49,167.248.133.53,0.0,NaN,nanYiB,CENSYS-ARIN-03,United States,False,100
57,57,111.7.96.166,0.0,NaN,nanYiB,Guangdong Mobile Communication Co.Ltd.,China,False,100
62,62,184.105.139.69,0.0,NaN,nanYiB,HURRICANE,United States,False,100
64,64,65.49.20.69,0.0,NaN,nanYiB,HURRICANE,United States,False,100
95,95,74.120.14.40,0.0,NaN,nanYiB,CENSYS-ARIN-02,United States,False,100
114,114,74.120.14.39,0.0,NaN,nanYiB,CENSYS-ARIN-02,United States,False,100
138,138,162.142.125.128,0.0,NaN,nanYiB,CENSYS-ARIN-01,United States,False,100
142,142,184.105.247.195,0.0,NaN,nanYiB,HURRICANE,United States,False,100


## Tor Nodes communications

In [92]:
%%time
df_tor = df[["srcip","dstip"]].compute()

Wall time: 24.2 s


In [103]:
%%time
unique_ips_src = df_tor['srcip'].unique()
unique_ips_dst = df_tor['dstip'].unique()

unique_ips_src = pd.Series(unique_ips_src, index = unique_ips_src)
unique_ips_dst = pd.Series(unique_ips_dst, index = unique_ips_dst)

df_tor['is_tor_node_src'] = df_tor['srcip'].map(unique_ips_src.apply(is_tor_node))
df_tor['is_tor_node_dst'] = df_tor['dstip'].map(unique_ips_dst.apply(is_tor_node))

Wall time: 14.5 s


In [104]:
df_tor[df_tor["is_tor_node_dst"]]["dstip"].value_counts()

51.15.47.139      157
46.105.55.153       8
83.135.89.183       6
82.65.125.114       6
185.21.216.195      6
188.126.94.37       3
188.40.142.18       1
Name: dstip, dtype: int64

In [105]:
df_tor[df_tor["is_tor_node_src"]]["srcip"].value_counts()

45.129.56.200      141
185.220.100.240     90
171.25.193.78       85
185.220.100.252     70
185.220.100.243     55
185.220.101.144     40
185.220.101.6       33
205.185.116.3       26
142.44.246.156      22
185.220.100.250     20
84.209.139.0        19
162.247.74.204      19
199.195.251.84      16
37.187.196.70       12
185.220.100.246     11
163.172.213.212     11
185.220.102.251     11
185.220.100.242     11
198.144.120.177     10
107.189.10.42       10
209.141.55.26       10
77.247.181.163      10
185.220.101.7       10
185.220.101.136      9
199.195.250.77       9
185.83.214.69        9
51.210.80.127        9
185.220.102.4        8
176.10.104.240       8
5.2.69.50            8
185.220.101.142      8
185.220.100.255      8
185.220.100.247      8
18.27.197.252        8
89.163.154.91        7
89.163.249.244       4
185.220.101.146      3
64.113.32.29         2
185.220.101.137      1
185.220.101.1        1
Name: srcip, dtype: int64

## [TODO] Research communications by user

## [TODO] Upload logs to Timesketch

## [TODO] Use of ML to find communication anomalies